## Figure 7: Core inputs to oviIN
This file describes the steps to create the network plots shown in figure 7. They use the modularity data and pagerank data of each node.
1. Imports and modularity data
2. Filter dataframe to cluster 3 and includes the oviIN
3. Finds pagerank and color to important nodes
4. Plot of the FS1A Input
5. Same process done for the IB017 and the SMP052

In [7]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import bokeh
import bokeh.palettes
from pyvis.network import Network
import networkx as nx

In [8]:
from neuprint import Client
from neuprint import fetch_simple_connections, fetch_neurons, NeuronCriteria as NC


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + 'v1.2.1', token=auth_token)
except:
    np_client = None

In [9]:
# Load in modularity data
oviINr_bodyID = 423101189
ovi_in = pd.read_csv('oviIN/preprocessed_inputs-v1.2.1/preprocessed_nodes.csv')
ovi_in

,id,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,...,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
0,1003215282,1,1,1,1,1,1,1,CL229_R,CL229,...,Traced,False,Roughly traced,PDM19,301.0,"[23044, 14981, 11600]","{'INP': {'pre': 87, 'post': 351, 'downstream':...",NaN,"['EPA(R)', 'GOR(R)', 'IB', 'ICL(R)', 'INP', 'S...","['GOR(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S..."
1,1005952640,2,2,1,1,2,2,2,IB058_R,IB058,...,Traced,False,Roughly traced,PVL20,NaN,NaN,"{'INP': {'pre': 464, 'post': 1327, 'downstream...",NaN,"['ATL(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...","['ATL(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S..."
2,1006928515,3,1,1,1,3,3,3,CL300_R,CL300,...,Traced,False,Roughly traced,PVL13,236.0,"[12083, 10523, 16816]","{'INP': {'pre': 79, 'post': 126, 'downstream':...",NaN,"['ATL(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S...","['ATL(R)', 'IB', 'ICL(R)', 'INP', 'SCL(R)', 'S..."
3,1007260806,4,2,1,1,4,4,4,CL301_R,CL301,...,Traced,False,Roughly traced,PVL13,236.0,"[13524, 10108, 16480]","{'INP': {'pre': 40, 'post': 128, 'downstream':...",NaN,"['GOR(R)', 'IB', 'ICL(R)', 'INP', 'PLP(R)', 'S...","['IB', 'ICL(R)', 'INP', 'PLP(R)', 'SCL(R)', 'S..."
4,1008024276,5,3,2,2,5,5,5,FB5N_R,FB5N,...,Traced,False,Roughly traced,AVM08,472.5,"[19178, 29711, 37312]","{'SNP(L)': {'post': 5, 'upstream': 5, 'mito': ...",SMPCREFB5_4,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2506,987273073,2507,3,8,8,409,604,629,(PVL05)_L,None,...,Traced,False,Roughly traced,NaN,NaN,NaN,"{'SNP(R)': {'pre': 65, 'post': 52, 'downstream...",NaN,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'..."
2507,987842109,2508,3,9,23,533,780,815,None,None,...,Orphan,NaN,Orphan hotknife,NaN,NaN,NaN,"{'SNP(R)': {'pre': 2, 'post': 13, 'downstream'...",NaN,"['SMP(R)', 'SNP(R)']","['SMP(R)', 'SNP(R)']"
2508,988567837,2509,2,3,4,16,58,63,FB4G_R,FB4G,...,Traced,False,Roughly traced,AVM08,NaN,NaN,"{'SNP(R)': {'pre': 6, 'post': 73, 'downstream'...",CRELALFB4_3,"['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F..."
2509,988909130,2510,2,3,4,389,559,572,FB5V_R,FB5V,...,Traced,False,Roughly traced,AVM10,296.5,"[13226, 32024, 18600]","{'SNP(R)': {'pre': 1, 'post': 28, 'downstream'...",CRELALFB5,"['AB(R)', 'CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX',...","['CRE(-ROB,-RUB)(R)', 'CRE(R)', 'CX', 'FB', 'F..."


In [10]:
# import ovi pre-synaptic data
ovi_pre_syns = pd.read_csv('ovi_pre_syns.csv', index_col=0)
ovi_pre_syns

,bodyId_pre,bodyId_post,roi_pre,roi_post,x_pre,y_pre,z_pre,x_post,y_post,z_post,confidence_pre,confidence_post,type_pre,type_post
0,989228019,423101189,CRE(R),CRE(R),22857,34611,17485,22825,34624,17464,0.998,0.633171,FB5V,NaN
1,988909130,423101189,CRE(R),CRE(R),19523,31096,21707,19509,31112,21693,0.969,0.528840,FB5V,NaN
2,988567837,423101189,LAL(R),LAL(R),23100,27659,23824,23074,27659,23801,0.970,0.459446,FB4G,NaN
3,987842109,423101189,SMP(R),SMP(R),19041,33090,14598,19047,33072,14598,0.973,0.974019,NaN,NaN
4,987273073,423101189,CRE(R),CRE(R),19347,27826,15348,19351,27806,15376,0.943,0.595921,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21619,1006928515,423101189,IB,IB,22664,14840,19749,22651,14823,19757,0.994,0.865103,CL300,NaN
21620,1005952640,423101189,IB,IB,21998,15625,18990,22007,15603,18991,0.996,0.929040,IB058,NaN
21621,1005952640,423101189,IB,IB,22998,15275,23049,23014,15272,23060,0.835,0.780000,IB058,NaN
21622,1005952640,423101189,IB,IB,23034,15380,23131,23048,15364,23146,0.986,0.905853,IB058,NaN


In [57]:
def importance(mod_df, res, clu_id):
    """
    This function takes a cell type name, a modularity dataframe, and the resolutions.
    It returns the pagerank, in-degree, & in-degree centrality of the cell types in the cluster across resolutions.
    """
    # get the ids of the neurons in the cluster key
    mod_ids = mod_df[mod_df[res]==clu_id]['id'].tolist()

    # add oviIN bodyId to list
    mod_ids = mod_ids+[oviINr_bodyID]
    

    if len(mod_ids)<=1:
        pg = [[]]
        in_deg = [[]]
        in_deg_centr = [[]]
        return pg, in_deg, in_deg_centr
        #return '1 or fewer neurons in this cluster'
    
    # fetch simple connections among neurons in chosen cluster
    clu_connectome = fetch_simple_connections(mod_ids, mod_ids, min_weight=3)
    
    # replace None with string 'None' to allow it to be a node in the graph
    clu_connectome = clu_connectome.fillna('None')
    
    # group by celltype and count the number of connections
    clu_type_connectome = clu_connectome[['type_pre','type_post','weight']].groupby(['type_pre','type_post']).sum()
    
    # let's threshold?
    clu_type_connectome = clu_type_connectome[clu_type_connectome['weight']>1]
    
    # reset the index to make the dataframe easier to work with
    clu_type_connectome = clu_type_connectome.reset_index()
    
    # make a directed graph from the dataframe
    G = nx.from_pandas_edgelist(clu_type_connectome, 'type_pre', 'type_post', edge_attr='weight', create_using=nx.DiGraph())
    
    # sorted dictionary of the in-degrees of the nodes in the graph
    in_deg = G.in_degree(weight='weight')
    in_deg = sorted(dict(in_deg).items(), key=lambda x: x[1], reverse=True)

    # sorted dictionary of the in-degree centrality of the nodes in the graph
    in_deg_centr = nx.in_degree_centrality(G)
    in_deg_centr = sorted(in_deg_centr.items(), key=lambda x: x[1], reverse=True)

    # get the importance of the cell type in the cluster
    pg = nx.pagerank(G)
    pg = sorted(pg.items(), key=lambda x: x[1], reverse=True)
    
    return pg, in_deg, in_deg_centr
# the resolutions
resos = ['0.0','0.05','0.1','0.5','0.75','1.0']

### Coarse Cluster 3 process implementation

In [12]:
# Coarse cluster 3
cluster_3 = ovi_in[ovi_in['0.0'] == 3]

# Edit dataframe to use only resolutions and celltypes needed
cluster_3 = cluster_3[['id','0.0','0.05','0.1','0.5','0.75','1.0', 'celltype']]

# Add in oviIN row
cluster_3 = cluster_3.append(ovi_in.loc[ovi_in['celltype'] == 'oviIN'])
cluster_3

/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_36011/31787349.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster_3 = cluster_3.append(ovi_in.loc[ovi_in['celltype'] == 'oviIN'])


,id,0.0,0.05,0.1,0.5,0.75,1.0,celltype,key,instance,...,status,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois
4,1008024276,3,2,2,5,5,5,FB5N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1008369339,3,2,2,5,5,8,FS1B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1008391068,3,2,2,5,5,5,FC2B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1008693605,3,2,2,5,5,9,FC2C,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1009371535,3,2,2,5,5,10,FC2B,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2501,985839488,3,10,5,532,778,821,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2504,986974738,3,8,8,93,116,124,CRE010,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2506,987273073,3,8,8,409,604,629,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2507,987842109,3,9,23,533,780,815,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Get the page rank
# Specified row
cluster_row = cluster_3[cluster_3['celltype'] == 'FS1A']

# Make a dictionary of the resolutions column mode values
cluster_row_dict = cluster_row[resos].mode(axis=0).to_dict('records')[0]

# the res we want to look at
res = '0.75'

clu_id = cluster_row_dict[res]
pg, in_deg, in_deg_c = importance(cluster_3, res, clu_id)
print(res, clu_id, pg[0], in_deg[0], in_deg_c[0])

[1008024276, 1008369339, 1008391068, 1008693605, 1009371535, 1010437434, 1014102651, 1014223948, 1014457195, 1014461486, 1014465452, 1014465588, 1014465666, 1014474192, 1035393986, 1039063931, 1039068185, 1039085109, 1039400642, 1039426238, 1039754434, 1041071176, 1044800536, 1044805646, 1044918145, 1045141938, 1045268099, 1070103331, 1070422316, 1071121755, 1071496918, 1071760480, 1071769518, 1072507501, 1072779298, 1075598224, 1075611602, 1076185604, 1102804327, 1104155212, 1106888382, 1132168736, 1133562752, 1134849078, 1135898324, 1135941581, 1136252170, 1138026774, 1164545788, 1164882751, 1165305889, 1166674437, 1195891557, 1195999722, 1199327859, 1230095228, 1257629175, 1291142212, 1291374677, 1320480289, 1323454861, 1343891368, 1345061120, 1351065769, 1570111396, 298944444, 329983926, 451365099, 486842208, 514807275, 578936675, 5813011125, 5813014391, 5813014693, 5813016900, 5813021046, 5813021247, 5813022923, 5813041684, 5813055999, 5813057464, 5813060701, 5813061260, 581306536

In [14]:
# get the cluster id for our neuron of interest at the resolution of interest
clu_id = cluster_row_dict[res]

# grab the neurons in the cluster
clu_ids = ovi_in[ovi_in[res]==clu_id]['id']

# Add in the oviINr bodyID to clu_ids series

clu_ids = clu_ids.append(pd.Series([oviINr_bodyID]))
from neuprint import fetch_simple_connections

# get the connectivity among prominent inputs
conn_cluster = fetch_simple_connections(clu_ids, clu_ids, min_weight=3)
conn_cluster

/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_36011/4019828187.py:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clu_ids = clu_ids.append(pd.Series([oviINr_bodyID]))


,bodyId_pre,bodyId_post,weight,type_pre,type_post,instance_pre,instance_post,conn_roiInfo
0,423101189,701345660,151,oviIN,LAL182,oviIN_R,LAL182_R,"{'SNP(R)': {'pre': 142, 'post': 142}, 'SMP(R)'..."
1,423101189,5813117385,134,oviIN,MBON27,oviIN_R,MBON27(y5d)_R,"{'SNP(R)': {'pre': 90, 'post': 90}, 'SMP(R)': ..."
2,423101189,705463539,76,oviIN,FB5F,oviIN_R,FB5F_R,"{'SNP(R)': {'pre': 69, 'post': 69}, 'SMP(R)': ..."
3,735488299,423101189,67,LAL022,oviIN,LAL022_R,oviIN_R,"{'INP': {'pre': 35, 'post': 35}, 'CRE(R)': {'p..."
4,423101189,607933758,65,oviIN,FB5Q,oviIN_R,FB5Q_R,"{'SNP(R)': {'pre': 63, 'post': 63}, 'SMP(R)': ..."
...,...,...,...,...,...,...,...,...
2325,5813103626,1195999722,3,FC2C,FC2B,FC2C(FQ7)_C8_L,FC2B(FQ7)_C8_L,"{'CX': {'pre': 3, 'post': 3}, 'FB': {'pre': 3,..."
2326,5813117385,5813021046,3,MBON27,CRE102,MBON27(y5d)_R,CRE102_R,"{'INP': {'pre': 3, 'post': 3}, 'CRE(R)': {'pre..."
2327,5813129271,1072779298,3,FC2C,FC2C,FC2C(FQ7)_C8_L,FC2C(FQ7)_C2_L,"{'INP': {'pre': 2, 'post': 2}, 'CRE(R)': {'pre..."
2328,5813129271,1075611602,3,FC2C,FS1A,FC2C(FQ7)_C8_L,FS1A(FQ9)_C8_R,"{'CX': {'pre': 3, 'post': 3}, 'FB': {'pre': 3,..."


In [15]:
# replace None with string 'None' to allow it to be a node in the graph
conn_cluster = conn_cluster.fillna('None')

# group by celltype and count the number of connections
conn_types_cluster = conn_cluster[['type_pre','type_post','weight']].groupby(['type_pre','type_post']).sum()
conn_types_cluster

weight
type_pre type_post        
CRE102   FB2L           16
         MBON27          9
FB2L     FC2A           16
         FC2B          139
         FC2C           14
...                    ...
oviIN    FC2C          103
         LAL022         72
         LAL182        151
         MBON27        134
         SMP371          3

[272 rows x 1 columns]

In [16]:
# Thresholding
conn_types_cluster = conn_types_cluster[conn_types_cluster['weight']>30]

# reset the index to make the dataframe easier to work with
conn_types_cluster = conn_types_cluster.reset_index()
conn_types_cluster

,type_pre,type_post,weight
0,FB2L,FC2B,139
1,FB2L,FS1A,71
2,FB5AA,FC2B,160
3,FB5AA,FS1A,34
4,FB5AA,FS1B,44
...,...,...,...
90,oviIN,FC2B,132
91,oviIN,FC2C,103
92,oviIN,LAL022,72
93,oviIN,LAL182,151


In [17]:
# Attach color to FS1A node and oviIN node
conn_types_cluster['color'] = 'lightblue'
conn_types_cluster.loc[conn_types_cluster['type_pre'] == 'FS1A', 'color'] = 'red'
conn_types_cluster.loc[conn_types_cluster['type_pre'] == 'oviIN', 'color'] = 'green'


In [18]:
# Attach pagerank to each celltype in type_pre
pr = dict(pg)
# create color dict that maps cell types to color column
color = dict(zip(conn_types_cluster['type_pre'], conn_types_cluster['color']))


,type_pre,type_post,weight,color
0,FB2L,FC2B,139,lightblue
1,FB2L,FS1A,71,lightblue
2,FB5AA,FC2B,160,lightblue
3,FB5AA,FS1A,34,lightblue
4,FB5AA,FS1B,44,lightblue
...,...,...,...,...
90,oviIN,FC2B,132,green
91,oviIN,FC2C,103,green
92,oviIN,LAL022,72,green
93,oviIN,LAL182,151,green


In [19]:
color

{'FB2L': 'lightblue',
 'FB5AA': 'lightblue',
 'FB5C': 'lightblue',
 'FB5D': 'lightblue',
 'FB5E': 'lightblue',
 'FB5F': 'lightblue',
 'FB5G': 'lightblue',
 'FB5N': 'lightblue',
 'FB5O': 'lightblue',
 'FB5Q': 'lightblue',
 'FB6U': 'lightblue',
 'FB6V': 'lightblue',
 'FB6W': 'lightblue',
 'FC2A': 'lightblue',
 'FC2B': 'lightblue',
 'FC2C': 'lightblue',
 'FS1A': 'red',
 'FS1B': 'lightblue',
 'LAL022': 'lightblue',
 'OA-VPM3': 'lightblue',
 'SMP371': 'lightblue',
 'oviIN': 'green'}

In [20]:
pr

{'FS1A': 0.18582359757258368,
 'oviIN': 0.12488838809966807,
 'FC2B': 0.09232427037289713,
 'FS1B': 0.06149650894787352,
 'LAL182': 0.06113458866556821,
 'MBON27': 0.05025294904021101,
 'FB5Q': 0.04317186415906008,
 'FC2A': 0.03805828721611154,
 'FB5O': 0.03417867754801773,
 'FB5G': 0.03192398226636473,
 'FB5E': 0.03172741657802577,
 'FB2L': 0.030007130803268292,
 'FB5F': 0.028661093186856796,
 'FC2C': 0.027846254761570974,
 'CRE102': 0.02711590022497329,
 'FB5D': 0.024558910035686388,
 'SMP110': 0.016010979420364096,
 'PFL2': 0.014481029145990584,
 'FB5C': 0.01398459727421688,
 'FB5N': 0.012566523947132039,
 'OA-VPM3': 0.010209650871866115,
 'FB5AA': 0.00898533406605662,
 'SMP371': 0.008015234345928041,
 'FB6W': 0.007558671767598461,
 'FB6V': 0.0075434969257950875,
 'FB6U': 0.007474662756315088}

## Full functions

In [85]:
def viz_network(df, importantNode, figfileName, pr, color_dict):
  G = nx.from_pandas_edgelist(df, 'type_pre', 'type_post', edge_attr='weight',create_using=nx.DiGraph())
  # Function cell
  nt = Network('1000px', '1000px', notebook=True, cdn_resources='in_line')

  # add in nodes
  for node in G.nodes:
      if node == [importantNode, 'oviIN']:
          nt.add_node(node, size=pr.get(node, 0)*500, color=color_dict.get(node, 0), title='pagerank:'+str(pr.get(node, 0)), font='500px')
      nt.add_node(node, size=pr.get(node, 0)*500, title='pagerank:'+str(pr.get(node, 0)), color=color_dict.get(node, 'lightblue'))
  # add in edges
  for edge in G.edges:
      if edge[0]==edge[1]:
          nt.add_edge(edge[0], edge[1], value=G.edges[edge]['weight']/10, color='black')
      else:
          nt.add_edge(edge[0], edge[1], value=G.edges[edge]['weight']/10, color='gray')
  #nt.show_buttons(filter_=['nodes', 'edges', 'physics'])
  nt.set_options("""const options = {
  "nodes": {
    "borderWidth": null,
    "borderWidthSelected": null,
    "opacity": null,
    "font": {
      "size": 20
    },
    "physics": false,
    "size": null
  },
  "edges": {
    "arrows": {
      "to": {
        "enabled": true,
        "scaleFactor": 0.5
      }
    },
    "color": {
      "inherit": true
    },
    "selfReferenceSize": null,
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "forceDirection": "none"
    }
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2500,
      "avoidOverlap": 1
    },
    "minVelocity": 0.75
  }
}
  """)
  nt.show(figfileName + '.html')

# Function for the full implementation from modularity dataframe
def circuit_viz(df, clus_id, res, importantNode, filename, resos):
    # Coarse cluster 1
    cluster = df[df['0.0'] == clus_id]
    cluster = cluster[['id','0.0','0.05','0.1','0.5','0.75','1.0', 'celltype']]
    cluster = cluster.append(ovi_in.loc[ovi_in['celltype'] == 'oviIN'])

    # Specified row
    cluster_row = cluster[cluster['celltype'] == importantNode]

    # Make a dictionary of the resolutions column mode values
    cluster_row_dict = cluster_row[resos].mode(axis=0).to_dict('records')[0]

    clu_id = cluster_row_dict[res]
    pg, in_deg, in_deg_c = importance(cluster, res, clu_id)

    # get the cluster id for our neuron of interest at the resolution of interest
    clu_id = cluster_row_dict[res]

    # grab the neurons in the cluster
    clu_ids = df[df[res]==clu_id]['id']

    # Add in the oviINr bodyID to clu_ids series (just in case)
    clu_ids = clu_ids.append(pd.Series([oviINr_bodyID]))

    # get the connectivity among prominent inputs
    conn_cluster = fetch_simple_connections(clu_ids, clu_ids, min_weight=3)

    # replace None with string 'None' to allow it to be a node in the graph
    conn_cluster = conn_cluster.fillna('None')

    # group by celltype and count the number of connections
    conn_types_cluster = conn_cluster[['type_pre','type_post','weight']].groupby(['type_pre','type_post']).sum()

    # Thresholding
    conn_types_cluster = conn_types_cluster[conn_types_cluster['weight']>30]

    # reset the index to make the dataframe easier to work with
    conn_types_cluster = conn_types_cluster.reset_index()

    # Attach color to FS1A node and oviIN node
    conn_types_cluster['color'] = 'lightblue'
    conn_types_cluster.loc[conn_types_cluster['type_pre'] == importantNode, 'color'] = 'red'
    conn_types_cluster.loc[conn_types_cluster['type_pre'] == 'oviIN', 'color'] = 'green'

    # Attach pagerank to each celltype in type_pre
    pr = dict(pg)
    # create color dict that maps cell types to color column
    color = dict(zip(conn_types_cluster['type_pre'], conn_types_cluster['color']))
    viz_network(conn_types_cluster, importantNode, filename, pr, color)

In [84]:
# Visualize FS1A in cluster 3
circuit_viz(ovi_in, 3, '0.75', 'FS1A', 'FS1A_cluster3', resos)


/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_6516/2962315450.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(ovi_in.loc[ovi_in['celltype'] == 'oviIN'])
/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_6516/2962315450.py:81: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clu_ids = clu_ids.append(pd.Series([oviINr_bodyID]))


FS1A_cluster3.html


In [76]:
# Visualize IB017 in cluster 2
circuit_viz(ovi_in, 2, '0.75', 'IB017', 'IB017_cluster2', resos)

IB017_cluster2.html


/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_6516/2888383693.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(ovi_in.loc[ovi_in['celltype'] == 'oviIN'])
/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_6516/2888383693.py:82: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clu_ids = clu_ids.append(pd.Series([oviINr_bodyID]))


In [77]:
# Visualize SMP052 in cluster 1
circuit_viz(ovi_in, 1, '0.75', 'SMP052', 'SMP052_cluster1', resos)

/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_6516/2888383693.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(ovi_in.loc[ovi_in['celltype'] == 'oviIN'])
/var/folders/db/q8_kwk2x73df8d997nrf14k1nfx1_b/T/ipykernel_6516/2888383693.py:82: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clu_ids = clu_ids.append(pd.Series([oviINr_bodyID]))


SMP052_cluster1.html
